# Produtos Digitais

Este notebook apresenta o ciclo de vida de coleta e uso dos dados de aplicativos e portais. Os dados são gerados durante o uso (navegação) dos produtos e capturados na forma de eventos, via `Google Analytics` e/ou `Google Firebase`
<br><br>
Vamos falar:

- Motivação
- Captura de dados
- Acesso e formato dos dados
- Como os dados podem ser usados
- Primeiros passos técnicos desse trabalho

### Motivação

- Muitos parceiros (clientes) estão usando dados para potencializar seus produtos digitais
- Expandir a visão de todos sobre o que é possível fazer com os dados desses produtos
- Provocar discussões e fortalecer os grupos de debate sobre Growth

### Captura dos dados

Esse trabalho começa com a definição do Tag Book (<i>mapa de jornadas dos produto e suas respectivas tags</i>) e evolui para a implementação das regras de coleta no Google Tag Manager (GTM). O Analista de Dados faz esse trabalho em conjunto com o time de desenvolvimento e o time de negócio, pois muitas vezes é necessário planejar tarefas e encaixá-las no backlog do time de desenvolvimento.

<img src="images/google-analytics-raw-data.jpg" />

### Acesso e formato dos dados

In [1]:
from pyspark.sql import types as T
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

Neste exemplo vamos utilizar um dataset fictício para simular os eventos coletados de um aplicativo ou portal.

In [9]:
spark = SparkSession.builder.getOrCreate()

df = spark.read.option('header', True).csv('data.csv')

df = df.select([
    F.col('timestamp').cast(T.TimestampType()),
    F.col('user_id').cast(T.IntegerType()),
    F.col('event_name'),
    F.col('event_params')
])

df.show(30, truncate=False)

+-------------------+-------+-----------------+-------------------------------------------------------------------+
|timestamp          |user_id|event_name       |event_params                                                       |
+-------------------+-------+-----------------+-------------------------------------------------------------------+
|2021-08-01 11:00:00|1      |login            |null                                                               |
|2021-08-01 11:00:10|1      |home_screenview  |null                                                               |
|2021-08-01 11:01:00|1      |restaurant_choose|{'id':10,'brand':'Pizza Hut'}                                      |
|2021-08-01 11:02:00|1      |view_item        |{'item_id':22,'price':10.0,'category':'Pizza','promotion':'No'}    |
|2021-08-01 11:04:00|1      |add_to_cart      |{'item_id':22,'qnt':1}                                             |
|2021-08-01 11:05:00|1      |purchase         |{'order':100,'value':10.0

## Como os dados podem ser usados

A seguir, algumas das ações que podem ser feitas com 

- Aprender sobre o negócio e sobre os consumidores
  - Taxa de conversão
  - Percentual de abandono de carrinho
  - Predominância por tipo de pedido ou prato?
  - Quem pede no almoço também pede no jantar?
  - O valor médio dos pedidos é o mesmo para almoço e jantar?
  - Existe diferença no fim de semana?
  - O que é uma boa experiência? Promoção? Receber o produto no prazo esperado? Pagar com meio de pagamento desejado?
  - Os clientes que tem uma boa experiência voltam?
  - ...
- Identificar oportunidades para potencializar os resultados
  - Ranking de restaurantes evita que os consumidores peçam em estabelecimentos com baixa qualidade de serviço?
  - Recomendar restaurantes, pratos e promoções ajuda a fidelizar o consumidor e impulsionar vendas?
  - Predizer fraudes / consumidores mal intencionados fideliza o estabelecimento? Transmite segurança?
  - Prever atrasos na entrega ajudam com o alinhamento de expectativa do cliente?
  - É possível prever picos dos restaurantes e evitar os cancelamentos?
  - ...

## Primeiros passos técnicos desse trabalho

A primeira coisa a se fazer é criar o conceito de `sessão de uso`.

In [10]:
max_session_duration = 60 * 5

df_session = df.select([
    F.col('timestamp'),
    F.lag('timestamp', 1).over(Window.partitionBy('user_id').orderBy('timestamp')).alias('lag'),
    F.col('user_id'),
    F.col('event_name'),
    F.col('event_params')
])

df_session = df_session.withColumn('diff',
    df_session.timestamp.cast(T.LongType()) - df_session.lag.cast(T.LongType())
)

df_session = df_session.select([
    'timestamp',
    'lag',
    'user_id',
    'event_name',
    'event_params',
    'diff',
    F.when(df_session.diff < max_session_duration, F.lit(0)).otherwise(F.lit(1)).alias('session')
])

df_session = df_session.select([
    'timestamp',
    'lag',
    'user_id',
    'event_name',
    'event_params',
    'diff',
    'session',
    F.sum('session').over(Window.partitionBy(['user_id']).orderBy('timestamp')).alias('session_id')
]).orderBy(['user_id','session_id','timestamp'])

df_session.show(30, truncate=20)

+-------------------+-------------------+-------+-----------------+--------------------+----+-------+----------+
|          timestamp|                lag|user_id|       event_name|        event_params|diff|session|session_id|
+-------------------+-------------------+-------+-----------------+--------------------+----+-------+----------+
|2021-08-01 11:00:00|               null|      1|            login|                null|null|      1|         1|
|2021-08-01 11:00:10|2021-08-01 11:00:00|      1|  home_screenview|                null|  10|      0|         1|
|2021-08-01 11:01:00|2021-08-01 11:00:10|      1|restaurant_choose|{'id':10,'brand':...|  50|      0|         1|
|2021-08-01 11:02:00|2021-08-01 11:01:00|      1|        view_item|{'item_id':22,'pr...|  60|      0|         1|
|2021-08-01 11:04:00|2021-08-01 11:02:00|      1|      add_to_cart|{'item_id':22,'qn...| 120|      0|         1|
|2021-08-01 11:05:00|2021-08-01 11:04:00|      1|         purchase|{'order':100,'val...|  60|   

In [6]:
df_session.select([
    'timestamp',
    'user_id',
    'session_id',
    'event_name',
    'event_params'
]).orderBy(['user_id','session_id','timestamp']).show(50, truncate=50)

+-------------------+-------+----------+-----------------+--------------------------------------------------+
|          timestamp|user_id|session_id|       event_name|                                      event_params|
+-------------------+-------+----------+-----------------+--------------------------------------------------+
|2021-08-01 11:00:00|      1|         1|            login|                                              null|
|2021-08-01 11:00:10|      1|         1|  home_screenview|                                              null|
|2021-08-01 11:01:00|      1|         1|restaurant_choose|                     {'id':10,'brand':'Pizza Hut'}|
|2021-08-01 11:02:00|      1|         1|        view_item|{'item_id':22,'price':10.0,'category':'Pizza','...|
|2021-08-01 11:04:00|      1|         1|      add_to_cart|                            {'item_id':22,'qnt':1}|
|2021-08-01 11:05:00|      1|         1|         purchase|   {'order':100,'value':10.0,'delivery_value':2.0}|
|2021-08-0

In [7]:
total_sessions = df_session.select(['user_id','session_id']).distinct().count()
total_sessions

5

In [8]:
total_sessions_with_purchase = df_session \
    .select(['user_id','session_id']) \
    .filter(df_session.event_name == 'purchase') \
    .distinct() \
    .count()
total_sessions_with_purchase

2

In [9]:
# convertion rate
total_sessions_with_purchase / total_sessions

0.4